In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D,
    BatchNormalization, ReLU,
    GlobalAveragePooling2D, Dense, Add
)
from tensorflow.keras import Model

In [2]:
def inverted_residual_block(x, out_channels, stride, expansion):
    residual = x
    in_channels = x.shape[-1]

    # Expantion phase (1 x 1)
    if expansion != 1:
        x = Conv2D(
            filters=in_channels * expansion,
            kernel_size=1,
            padding='same',
            use_bias=False
        )(x)
        x = BatchNormalization()(x)
        x = ReLU(6.)(x)
        
    # Depthwise convolution (3 x 3)
    x = DepthwiseConv2D(
        kernel_size = 3,
        strides = stride,
        padding = 'same',
        use_bias = False
    )(x)

    x = BatchNormalization()(x)

    x = ReLU(6.)(x)

    # Projection phase (1 x 1)
    x = Conv2D(
        filters = out_channels,
        kernel_size = 1,
        padding = 'same',
        use_bias = False
    )(x)

    x = BatchNormalization()(x)

    if stride == 1 and in_channels == out_channels:
        x = Add()([residual, x])

    return x

In [3]:
def MobileNet(input_shape=(224, 224, 3), num_classes=100):
    inputs = Input(shape=input_shape)

    # Initial convolution (stem)
    x = Conv2D(
        filters = 32,
        kernel_size = 3,
        strides = 2,
        padding = 'same',
        use_bias = False
    )(inputs)

    x = BatchNormalization()(x)

    x = ReLU(6.)(x)


    # Block configuration
    config = [
        (1,  16, 1, 1),            # (expansion, output_channels, repeats, stride)
        (6,  24, 2, 2),
        (6,  32, 3, 2),
        (6,  64, 4, 2),
        (6,  96, 3, 1),
        (6, 160, 3, 2),
        (6, 320, 1, 1),
    ]

    # Build inverted residual blocks
    for expansion, channels, repeats, stride in config:
        for i in range(repeats):
            s = stride if i == 0 else 1
            x = inverted_residual_block(x, channels, s, expansion)

    # Final convolution
    x = Conv2D(
        filters = 1280,
        kernel_size = 1,
        padding = 'same',
        use_bias = False
    )(x)

    x = BatchNormalization()(x)

    x = ReLU(6.)(x)


    # Classification head
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

In [4]:
model = MobileNet()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 112, 112, 32)      │             128 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu (ReLU)                  │ (None, 112, 112, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d              │ (None, 112, 112, 32)      │             288 │ re_lu[0][0]                │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 112, 112, 32)      │             128 │ depthwise_conv2d[0][0]     │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_1 (ReLU)                │ (None, 112, 112, 32)      │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 112, 112, 16)      │             512 │ re_lu_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 112, 112, 16)      │              64 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 112, 112, 96)      │           1,536 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 112, 112, 96)      │             384 │ conv2d_2[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_2 (ReLU)                │ (None, 112, 112, 96)      │               0 │ batch_normalization_3[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ depthwise_conv2d_1            │ (None, 56, 56, 96)        │             864 │ re_lu_2[0][0]              │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_4         │ (None, 56, 56, 96)        │             384 │ depthwise_conv2d_1[0][0]   │
│ (BatchNormalization)          │                           │               

 Total params: 2,386,084 (9.10 MB)

 Trainable params: 2,351,972 (8.97 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [5]:
# tf.keras.utils.plot_model(
#     model,
#     to_file="mobilenetv2_architecture.png",
#     show_shapes=True,
#     show_layer_names=True
# )